## 🌿 NDVI Computation from Drone Imagery

This code calculates the **Normalized Difference Vegetation Index (NDVI)** using:

- **Red band**: `IMG_0666_3.TIF`
- **Near-Infrared (NIR) band**: `IMG_0666_4.TIF`

### 📘 NDVI Formula
$$
\text{NDVI} = \frac{\text{NIR} - \text{Red}}{\text{NIR} + \text{Red}}
$$

### 🧪 Processing Steps
- Reads Red and NIR bands using `rasterio`
- Computes NDVI and clips values to the range [-1, 1]
- Visualizes NDVI using a green-to-red colormap (`RdYlGn`)

### 🎨 Output
- **High NDVI (green)**: Healthy vegetation
- **Low NDVI (red/yellow)**: Bare soil, built-up areas, or stressed vegetation

### 🧠 Student Prompts
- What does a high NDVI value indicate about vegetation health?
- How might NDVI vary across different land cover types?

In [3]:
# 🛰️ Remote Sensing & GIS Essentials with Python
# Developed for Ravi’s hydrological and environmental modeling tools
# Includes: Raster I/O, NDVI, Clipping, Zonal Stats, Folium Mapping

!pip install rasterio geopandas matplotlib folium shapely pyproj rasterstats --quiet

import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import folium
import json
from rasterio.mask import mask
from rasterstats import zonal_stats


In [4]:
raster_path = 'LAB4.tif'  # NIR band

with rasterio.open(raster_path) as src:
    band = src.read(1)

plt.imshow(band, cmap='gray')
plt.title('Landsat Band 4 (NIR)')
plt.colorbar()
plt.show()


RasterioIOError: LAB4.tif: No such file or directory

In [ ]:
with rasterio.open('LAB4.tif') as nir_src:
    nir = nir_src.read(1).astype('float32')
with rasterio.open('LAB3.tif') as red_src:
    red = red_src.read(1).astype('float32')

ndvi = (nir - red) / (nir + red)

plt.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
plt.title('NDVI')
plt.colorbar()
plt.show()


In [ ]:
gdf = gpd.read_file('testshape.shp')
gdf.plot(edgecolor='black', facecolor='none')
plt.title('Study Area Boundary')
plt.show()


In [ ]:
shapes = [json.loads(gdf.to_json())['features'][0]['geometry']]

with rasterio.open('LAB4.tif') as src:
    out_image, out_transform = mask(src, shapes, crop=True)
    clipped_meta = src.meta.copy()
    clipped_meta.update({
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

plt.imshow(out_image[0], cmap='gray')
plt.title('Clipped Raster')
plt.show()


In [5]:
import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterstats import zonal_stats

# ---- 1. Export NDVI to a temporary GeoTIFF ----
# Load metadata from a reference raster (e.g. Red or NIR band)
with rasterio.open('LAB4.tif') as src:
    meta = src.meta.copy()
    meta.update(driver='GTiff', dtype=rasterio.float32, count=1)

    # Save NDVI array to new GeoTIFF
    with rasterio.open('ndvi.tif', 'w', **meta) as dst:
        dst.write(ndvi.astype(rasterio.float32), 1)

# ---- 2. Run Zonal Statistics ----
stats = zonal_stats('testshape.shp', 'ndvi.tif', stats=['mean', 'min', 'max'])
print("Zonal Statistics (NDVI):", stats)


RasterioIOError: LAB4.tif: No such file or directory